In [ ]:
from ngsolve import *
import numpy as np
import scipy
import scipy.sparse as sparse
import math
from ngsolve.webgui import Draw


In [ ]:

geo = SplineGeometry()

r1 = 2.5
r2 = 1
h1 = 5
h2 = 1.5
h3 = 1
d = 0.2


x_val = [0, 2*r1, 2*r1, r1+r2, r1+r2, r1+r2-d, r1+r2-d, 2*r1-d, 2*r1-d, d, d, r1-r2+d, r1-r2+d, r1-r2, r1-r2, 0]
y_val = [0, 0, h1, h1+h2, h1+h2+h3, h1+h2+h3, h1+h2, h1, d, d, h1, h1+h2, h1+h2+h3, h1+h2+h3, h1+h2, h1]

assert len(x_val)==len(y_val), "lengths of x and y values must be the same"
points = [None]*len(x_val)
amount_points = len(points)
for j in range(0,amount_points):
    points[j] = geo.AppendPoint(x_val[j], y_val[j])

for j in range(0,amount_points):
    geo.Append(["line", points[j], points[(j+1) % amount_points]], leftdomain=2, rightdomain=1, bc="neumann")

geo.AddRectangle((-1,-1),(2*r1+1, h1+h2+h3+1), leftdomain=1, bcs=["robin","robin","robin","robin"])
geo.SetMaterial (1, "air")
geo.SetMaterial (2, "glass")

mesh = Mesh(geo.GenerateMesh(maxh=0.2))
Draw(mesh)




In [ ]:
from ngsolve import *
import netgen.gui
from netgen.geom2d import *
geo = SplineGeometry()

## draw shark skin
x_val = [0,0.5,2,3.8,6,5.5,8.5,10,11,11,11.5,10,6.5,6,4.5,2.5,1]
y_val = [0,-0.5,-1,-1.2,-2.5,-1.5,-0.5,-0.25,-1,0.5,1.5,0.25,1,2,1,1,0.5]

assert len(x_val)==len(y_val), "lengths of x and y values must be the same"
points = [None]*len(x_val)
amount_points = len(points)
for j in range(0,amount_points):
    points[j] = geo.AppendPoint(x_val[j], y_val[j])

neumann_rb = {0,1,3,4,5,7,8,9,10,11,12,13,14,16}
for j in range(0,amount_points):
    if ((j+1) % amount_points in neumann_rb) and (j in neumann_rb):
        geo.Append(["line", points[j], points[(j+1) % amount_points]], leftdomain=1, rightdomain=0, bc="neumann")
    else:
        geo.Append(["line", points[j], points[(j+1) % amount_points]], leftdomain=1, rightdomain=0, bc="dirichlet")

## draw eye (source 1)
geo.AddCircle((2,0), 0.3, leftdomain=2, rightdomain=1)
geo.SetMaterial (1, "body")
geo.SetMaterial (2, "eye")

mesh = Mesh(geo.GenerateMesh(maxh=0.2))
Draw(mesh)

Sigma = HDiv(mesh, order=2, dirichlet="neumann")
V = L2(mesh, order=1)
X = FESpace([Sigma,V])

sigma, u = X.TrialFunction()
tau, v = X.TestFunction()

a = BilinearForm(X)
a += (sigma*tau + div(sigma)*v + div(tau)*u)*dx
a.Assemble()


## for u=0 on dirichlet boundaries
f = LinearForm(X)
# how do I do that with new syntax?
f += (10 * v) * dx (definedon=mesh.Materials("body"))
f += (-300*v) * dx (definedon=mesh.Materials("eye"))
f.Assemble()

gfu = GridFunction(X)
gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec
Draw (gfu.components[0], mesh, "flux")
Draw (gfu.components[1], mesh, "u")